# Objective

This experiment checks the following for a simple 2 layer FC network on MNIST.
1. Verify Agop and NFM relations for the conv layers
2. Run RFM to construct similar matrices as the above.(TBD)

The model is taken from MNIST/model3

# Setup

In [ ]:
!git clone --branch master_colab https://github.com/aravindadithya/DLR

In [ ]:
!pip install visdom
#python -m visdom.server -port 8097

In [3]:
!pip install hickle

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 2.3 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 31.3 MB/s eta 0:00:0000:0100:01


In [3]:
import sys
import os
current_dir = os.getcwd()
print(current_dir)

parent_dir = os.path.dirname(os.path.dirname(current_dir))
model_dir1 = os.path.join(parent_dir, 'trained_models', 'MNIST', 'model1', 'nn_models/')
model_dir3 = os.path.join(parent_dir, 'trained_models', 'MNIST', 'model3', 'nn_models/')

'''
parent_dir='C:\\Users\\garav\\AGOP\\DLR'
model_dir1= 'C:\\Users\\garav\\AGOP\\DLR\\trained_models\\MNIST\\model1\\nn_models\\'
model_dir3= 'C:\\Users\\garav\\AGOP\\DLR\\trained_models\\MNIST\\model3\\nn_models\\'
#parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
'''
sys.path.append(parent_dir)
print(f"Parent directory: {parent_dir}")
print(f"Model directory 1: {model_dir1}")
print(f"Model directory 3: {model_dir3}")

C:\Users\garav\AGOP\DLR\experiments\rfm_mnist
Parent directory: C:\Users\garav\AGOP\DLR
Model directory 1: C:\Users\garav\AGOP\DLR\trained_models\MNIST\model1\nn_models/
Model directory 3: C:\Users\garav\AGOP\DLR\trained_models\MNIST\model3\nn_models/


In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
from utils import agop_fc as af
from utils import agop_fc1 as af1
from torch.utils.data import Dataset
import random
import torch.backends.cudnn as cudnn
import rfm
import random
import torch.backends.cudnn as cudnn
from trained_models.MNIST.model1 import trainer as t1
from trained_models.MNIST.model3 import trainer as t3
import numpy as np
from sklearn.model_selection import train_test_split
from torch.linalg import norm
from torchvision import models
import torch.nn as nn
from copy import deepcopy
import torch.nn.functional as F

Setting up a new session...
Without the incoming socket you cannot receive events from the server or register event handlers to your Visdom client.


/workspace/DLR/experiments/rfm_mnist
/workspace/DLR
/workspace/DLR/trained_models/MNIST/model1/nn_models/
/workspace/DLR/experiments/rfm_mnist
/workspace/DLR
/workspace/DLR/trained_models/MNIST/model3/nn_models/


In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
#device='cpu'
print(f"Using device: {device}")

Using device: cuda:0


In [4]:
torch.cuda.empty_cache()

In [5]:
trainloader, valloader, testloader = t1.get_loaders()
net= t1.get_untrained_net()
init_net= deepcopy(net)
import os
if os.path.exists(model_dir1+'mnist_fc_trained_nn.pth'):
    checkpoint = torch.load(model_dir1+'mnist_fc_trained_nn.pth', map_location=torch.device(device))
    net.load_state_dict(checkpoint['state_dict'])  # Access the 'state_dict' within the loaded dictionary
    print("Model weights loaded successfully.")
else:
    print("Train network first")

#print("Train the network first")
#t1.train_net()

torch.Size([5000, 3, 32, 32])
torch.Size([5000, 3, 32, 32])
torch.Size([5000, 3, 32, 32])
torch.Size([5000, 3, 32, 32])
torch.Size([5000, 3, 32, 32])
torch.Size([5000, 3, 32, 32])
torch.Size([5000, 3, 32, 32])
torch.Size([5000, 3, 32, 32])
torch.Size([5000, 3, 32, 32])
torch.Size([5000, 3, 32, 32])
Train Size:  40000 Val Size:  10000
Test Size:  10000
torch.Size([900, 3, 32, 32])
torch.Size([900, 3, 32, 32])
torch.Size([900, 3, 32, 32])
torch.Size([900, 3, 32, 32])
torch.Size([900, 3, 32, 32])
torch.Size([892, 3, 32, 32])
torch.Size([900, 3, 32, 32])
torch.Size([900, 3, 32, 32])
torch.Size([900, 3, 32, 32])
torch.Size([900, 3, 32, 32])
Model weights loaded successfully.


In [8]:
trainloader, valloader, testloader = t3.get_loaders()
net= t3.get_untrained_net()
init_net= deepcopy(net)
import os
if os.path.exists(model_dir3+'mnist_fc_trained_nn.pth'):
    checkpoint = torch.load(model_dir3+'mnist_fc_trained_nn.pth', map_location=torch.device(device))
    net.load_state_dict(checkpoint['state_dict'])  # Access the 'state_dict' within the loaded dictionary
    print("Model weights loaded successfully.")
else:
    print("Train the network first")

torch.Size([5000, 3, 32, 32])
torch.Size([5000, 3, 32, 32])
torch.Size([5000, 3, 32, 32])
torch.Size([5000, 3, 32, 32])
torch.Size([5000, 3, 32, 32])
torch.Size([5000, 3, 32, 32])
torch.Size([5000, 3, 32, 32])
torch.Size([5000, 3, 32, 32])
torch.Size([5000, 3, 32, 32])
torch.Size([5000, 3, 32, 32])
Train Size:  40000 Val Size:  10000
Test Size:  10000
torch.Size([900, 3, 32, 32])
torch.Size([900, 3, 32, 32])
torch.Size([900, 3, 32, 32])
torch.Size([900, 3, 32, 32])
torch.Size([900, 3, 32, 32])
torch.Size([892, 3, 32, 32])
torch.Size([900, 3, 32, 32])
torch.Size([900, 3, 32, 32])
torch.Size([900, 3, 32, 32])
torch.Size([900, 3, 32, 32])
Model weights loaded successfully.


# Verify NFA for FC layers:

In [16]:

print(net)
af.verify_NFA(net, init_net, trainloader, max_batch= 50, classes=10, chunk_idx=1, layer_idx=1)

Net(
  (features): Sequential(
    (0): Linear(in_features=3072, out_features=1024, bias=False)
    (1): Nonlinearity()
    (2): Linear(in_features=1024, out_features=1024, bias=False)
    (3): Nonlinearity()
  )
  (classifier): Sequential(
    (0): Linear(in_features=1024, out_features=10, bias=False)
  )
)
Correlation between Initial and Trained CNFM:  tensor(0.4899, device='cuda:0')
Computing GOP for sample 0 out of 50
Computing GOP for sample 1 out of 50
Computing GOP for sample 2 out of 50
Computing GOP for sample 3 out of 50
Computing GOP for sample 4 out of 50
Computing GOP for sample 5 out of 50
Computing GOP for sample 6 out of 50
Computing GOP for sample 7 out of 50
Computing GOP for sample 8 out of 50
Computing GOP for sample 9 out of 50
Computing GOP for sample 10 out of 50
Computing GOP for sample 11 out of 50
Computing GOP for sample 12 out of 50
Computing GOP for sample 13 out of 50
Computing GOP for sample 14 out of 50
Computing GOP for sample 15 out of 50
Computing GOP

tensor([[ 0.1521,  0.0036,  0.0123,  ..., -0.0081, -0.0019,  0.0023],
        [ 0.0036,  0.1669, -0.0310,  ...,  0.0210, -0.0045,  0.0172],
        [ 0.0123, -0.0310,  0.1918,  ..., -0.0199, -0.0021, -0.0093],
        ...,
        [-0.0081,  0.0210, -0.0199,  ...,  0.1433,  0.0097,  0.0215],
        [-0.0019, -0.0045, -0.0021,  ...,  0.0097,  0.1614,  0.0088],
        [ 0.0023,  0.0172, -0.0093,  ...,  0.0215,  0.0088,  0.1550]],
       device='cuda:0')

In [12]:
af1.verify_NFA(net, init_net, trainloader, batch_size= 800, cutoff=10, chunk_idx=1, layer_idx=1)

torch.Size([1024, 1024])
Correlation between Initial and Trained CNFM:  tensor(0.4899, device='cuda:0', dtype=torch.float64)
Sequential(
  (0): Linear(in_features=3072, out_features=1024, bias=False)
  (1): Nonlinearity()
)
Computing Jacobian for batch:  0 50
Computing Jacobian for batch:  1 50
Computing Jacobian for batch:  2 50
Computing Jacobian for batch:  3 50
Computing Jacobian for batch:  4 50
Computing Jacobian for batch:  5 50
Computing Jacobian for batch:  6 50
Computing Jacobian for batch:  7 50
Computing Jacobian for batch:  8 50
Computing Jacobian for batch:  9 50
Computing Jacobian for batch:  10 50
Computing Jacobian for batch:  11 50
torch.Size([9600, 10, 1024])
0 12
1 12
2 12
3 12
4 12
5 12
6 12
7 12
8 12
9 12
10 12
11 12
Computing Jacobian for batch:  0 50
Computing Jacobian for batch:  1 50
Computing Jacobian for batch:  2 50
Computing Jacobian for batch:  3 50
Computing Jacobian for batch:  4 50
Computing Jacobian for batch:  5 50
Computing Jacobian for batch:  6 50

In [ ]:
#TODO: How to meaningfully visualise? 

# RFM 

In [ ]:
'''Warning: This is an extremely cpu intensive process since it uses solve function from linalg 
The rfm.py from utils is equipped with more memory efficient solvers. 
'''

rfm.rfm(trainloader, valloader, testloader, name=name,
            batch_size=10, iters=1, reg=1e-3)

# Debug

In [8]:
import numpy as np
import torch
#from numpy.linalg import solve
from torch.linalg import solve
import experiments.rfm_mnist.kernels as kernels #Should be called from utils
from tqdm import tqdm
import hickle
import torch.distributions as distributions

def laplace_kernel_M(pair1, pair2, bandwidth, M):
    return kernels.laplacian_M(pair1, pair2, bandwidth, M)


def get_grads(X, sol, L, P, batch_size=2):
    torch.cuda.empty_cache()
    M = 0.

    num_samples = 10000
    #indices = np.random.randint(len(X), size=num_samples)
    indices = torch.randint(0, len(X), (num_samples,),device=device)

    if len(X) > len(indices):
        x = X[indices, :]
    else:
        x = X

    K = laplace_kernel_M(X, x, L, P)

    dist = kernels.euclidean_distances_M(X, x, P, squared=False)
    
    dist = torch.where(dist < 1e-10, torch.zeros(1,device=device).float(), dist)

    K = K/dist
    K[K == float("Inf")] = 0.

    #a1 = torch.from_numpy(sol.T).float()
    a1= sol.T
    n, d = X.shape
    n, c = a1.shape
    m, d = x.shape

    a1 = a1.reshape(n, c, 1)
    X1 = (X @ P).reshape(n, 1, d)
    step1 = a1 @ X1
    del a1, X1
    step1 = step1.reshape(-1, c*d)

    step2 = K.T @ step1
    del step1

    step2 = step2.reshape(-1, c, d)

    #a2 = torch.from_numpy(sol).float()
    a2 = sol
    step3 = (a2 @ K).T

    del K, a2

    step3 = step3.reshape(m, c, 1)
    x1 = (x @ P).reshape(m, 1, d)
    step3 = step3 @ x1

    G = (step2 - step3) * -1/L

    M = 0.

    bs = batch_size
    batches = torch.split(G, bs)
    for i in tqdm(range(len(batches))):
        grad = batches[i].cuda()
        gradT = torch.transpose(grad, 1, 2)
        M += torch.sum(gradT @ grad, dim=0).cpu()
        del grad, gradT
    torch.cuda.empty_cache()
    M /= len(G)
    #M = M

    return M

def sample_normal(covariance_matrix_M, num_rows_k, matrix_dim_t):
   
    # Create a mean vector of zeros, compatible with the dimensions of M
    dtype = covariance_matrix_M.dtype
    mean_vector = torch.zeros(matrix_dim_t, dtype=dtype, device=device)
    mean_vector.to(device)
    #covariance_matrix_M.to(device)
    # Create a multivariate normal distribution object
    # torch.distributions.MultivariateNormal expects a covariance_matrix.
    # It internally checks for positive definiteness.
    try:
        mvn = distributions.MultivariateNormal(loc=mean_vector, covariance_matrix=covariance_matrix_M)
    except Exception as e:
        print(f"Error creating MultivariateNormal distribution: {e}")
        print("Please ensure your covariance_matrix_M is symmetric positive definite.")
        # Attempt to make it PSD for robustness in example, by adding a small diagonal perturbation
        # In a real scenario, you'd ensure your input M is correctly formed.
        min_eigval = torch.min(torch.linalg.eigvalsh(covariance_matrix_M)).item()
        if min_eigval <= 0:
            print(f"Warning: Covariance matrix has non-positive eigenvalue ({min_eigval:.2e}). Adding jitter.")
            covariance_matrix_M = covariance_matrix_M + torch.eye(matrix_dim_t, device=device, dtype=dtype) * (abs(min_eigval) + 1e-6)
            mvn = distributions.MultivariateNormal(loc=mean_vector, covariance_matrix=covariance_matrix_M)


    # Sample num_rows_k times from the distribution
    # The .sample() method will return a tensor of shape (num_rows_k, matrix_dim_t)
    sampled_matrix = mvn.sample((num_rows_k,))

    return sampled_matrix

def rfm(train_loader, val_loader, test_loader,
        iters=3, name=None, batch_size=2, reg=1e-3,
        train_acc=True):

    L = 10

    X_train, y_train = get_data(train_loader)
    X_val, y_val = get_data(val_loader)
    X_test, y_test = get_data(test_loader)
    
    # Trimming: Ideally should be avoided
    X_train, y_train = X_train[:30000], y_train[:30000]
    X_val, y_val = X_val[:10000], y_val[:10000]
    X_test, y_test = X_test[:10000], y_test[:10000]
    
    n, d = X_train.shape
    
    M = torch.eye(d, dtype=torch.float32)
    
    '''
    X_train= X_train.to(device)
    y_train= y_train.to(device)
    #X_val.to(device)
    #y_val.to(device)
    x_test= X_test.to(device)
    y_test= y_test.to(device)
    '''
    for i in range(iters):
        print(i)
        torch.cuda.empty_cache()
        K_train = laplace_kernel_M(X_train.to(device), X_train.to(device), L, M.to(device))
        #K_train.to(device)
        source = (K_train + reg * torch.eye(len(K_train), device=device)).float()

        
        sol = solve(source, y_train.to(device)).T
        sol = sol.to(device)
        if train_acc:
            y_pred = (sol @ K_train).T
            #y_pred = torch.from_numpy(preds)
            preds = torch.argmax(y_pred, dim=-1)
            labels = torch.argmax(y_train, dim=-1)
            count = torch.sum(labels.to(device) == preds.to(device))
            print("Round " + str(i) + " Train Acc: ", count / len(labels))

        K_test = laplace_kernel_M(X_train.to(device), X_test.to(device), L, M.to(device))
        y_pred = (sol @ K_test).T
        #print("Round " + str(i) + " MSE: ", torch.mean(torch.square(preds - y_test.numpy())))
        print("Round " + str(i) + " MSE: ", F.mse_loss(y_pred.to(device), y_test.to(device)))
        #y_pred = torch.from_numpy(preds)     
        preds = torch.argmax(y_pred, dim=-1)
        preds = preds.to(device)
        labels = torch.argmax(y_test, dim=-1)
        labels = labels.to(device)
        count = torch.sum(labels == preds)
        print("Round " + str(i) + " Acc: ", count / len(labels))

        M  = get_grads(X_train.to(device), sol.to(device), L, M.to(device), batch_size=batch_size)
        print(M.shape)
        if name is not None:
            hickle.dump(M, 'M_' + name + '_' + str(i) + '.h')
    W = sample_normal(M.to(device), 1024,3072)
    print(W)
    print(W.shape)
    print(net.features[0].weight)
    
    
    '''
    K_train = laplace_kernel_M(X_train, X_train, L, torch.from_numpy(M)).numpy()
    sol = solve(K_train + reg * np.eye(len(K_train)), y_train).T
    K_test = laplace_kernel_M(X_train, X_test, L, torch.from_numpy(M)).numpy()
    preds = (sol @ K_test).T
    mse = np.mean(np.square(preds - y_test.numpy()))
    print("Final MSE: ", mse)
    y_pred = torch.from_numpy(preds)
    preds = torch.argmax(y_pred, dim=-1)
    labels = torch.argmax(y_test, dim=-1)
    count = torch.sum(labels == preds).numpy()
    print(" Final Acc: ", count / len(labels))
    '''
    #return mse


def get_data(loader):
    X = []
    y = []
    for idx, batch in enumerate(loader):
        inputs, labels = batch
        X.append(inputs)
        y.append(labels)
    return torch.cat(X, dim=0), torch.cat(y, dim=0)



In [9]:
rfm(trainloader, valloader, testloader, name='fc_rfm',
            batch_size=10, iters=10, reg=1e-3)

0


OutOfMemoryError: CUDA out of memory. Tried to allocate 3.35 GiB. GPU 0 has a total capacity of 15.48 GiB of which 3.20 GiB is free. Including non-PyTorch memory, this process has 12.27 GiB memory in use. Of the allocated memory 11.53 GiB is allocated by PyTorch, and 413.72 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [116]:
print(net.features[0].weight.shape)
print(net.features[0].weight)

torch.Size([1024, 3072])
Parameter containing:
tensor([[-0.0164,  0.0110,  0.0087,  ...,  0.0010,  0.0019,  0.0024],
        [-0.0033, -0.0127,  0.0155,  ...,  0.0079, -0.0117, -0.0122],
        [-0.0145, -0.0007,  0.0079,  ..., -0.0125,  0.0094, -0.0095],
        ...,
        [ 0.0080,  0.0042, -0.0011,  ..., -0.0018,  0.0047, -0.0160],
        [-0.0113,  0.0027,  0.0152,  ...,  0.0084, -0.0130,  0.0136],
        [-0.0171, -0.0143,  0.0057,  ..., -0.0158,  0.0157,  0.0007]],
       requires_grad=True)


In [106]:
print(net)


Net(
  (features): Sequential(
    (0): Linear(in_features=3072, out_features=1024, bias=False)
    (1): Nonlinearity()
    (2): Linear(in_features=1024, out_features=1024, bias=False)
    (3): Nonlinearity()
  )
  (classifier): Sequential(
    (0): Linear(in_features=1024, out_features=10, bias=False)
  )
)
